# Chapter 10

## 10.1
java.awt.Rectangle类有两个很有用的方法translate和grow,但可惜的是像java.awt.geom.Ellipse2D这样的类没有。在Scala中，你可以解决掉这个问题。定义一个RectangleLike特质,加入具体的translate和grow方法。提供任何你需要用来实现的抽象方法,以便你可以像如下代码这样混入该特质:
```scala
val egg = new java.awt.geom.Ellipse2D.Double(5,10,20,30) with RectangleLike
egg.translate(10,-10)
egg.grow(10,20)
```
使用自身类型使得trait可以操作x,y

In [4]:
import java.awt.geom.Ellipse2D

trait RectangleLike {
    this: Ellipse2D.Double =>
    def translate(x:Double, y:Double) {
        this.x = x;
        this.y = y;
    }
    
    def grow(x:Double, y:Double) {
        this.x += x
        this.y += y
    }
}

val egg = new Ellipse2D.Double(5,10,20,30) with RectangleLike
egg.translate(10, -10)
egg.grow(10, 20)
println(egg.x + ", " + egg.y)

20.0, 10.0


import java.awt.geom.Ellipse2D


defined trait RectangleLike
egg: Ellipse2D.Double with wrapper.wrapper.RectangleLike = $sess.cmd3Wrapper$Helper$$anon$1@19860000

## 10.2
通过把scala.math.Ordered[Point]混入java.awt.Point的方式, 定义OrderedPoint类, 按辞典方式排序, 也就是说, 如果x < x' 或者x = x'且 y < y' 则 (x,y) < (x', y')

In [7]:
import java.awt.Point

class OrderedPoint(x: Int, y: Int) extends Point(x, y) with Ordered[Point] {
    def compare(that: Point): Int = {
        if (this.x <= that.x && this.y < that.y) -1
        else if (this.x == that.x && this.y == that.y) 0
        else 1
    }
}

println (new OrderedPoint(1, 2) > new OrderedPoint(0, 1))

true


import java.awt.Point


defined class OrderedPoint

## 10.3
查看BitSet类,将它的所有超类和特质绘制成一张图, 忽略类型参数, 然后给出该特质的线性化规格说明

## 10.4
提供一个CryptoLogger类, 将日志消息以凯撒密码加密, 缺省情况下密钥为3, 不过使用者也可以重写它, 提供缺省密钥和-3作为密钥时的使用示例

In [9]:
trait Logger {
    def log(str: String, key: Int = 3):String
}

class CryptoLogger extends Logger {
    def log(str: String, key: Int): String = {
        for (i <- str) yield (97 + (i - 97 + key + (if (key < 0) 26 else 0)) % 26).toChar
    }
}

val plain = "shizhenyu";
println("明文为：" + plain);
println("加密后为：" + new CryptoLogger().log(plain));
println("加密后为：" + new CryptoLogger().log(plain, -3));

明文为：shizhenyu
加密后为：vklckhqbx
加密后为：pefwebkvr


defined trait Logger
defined class CryptoLogger
plain: String = "shizhenyu"

## 10.5
JavaBean规范里有一种提法叫做属性变更监听器(property change listener)，这是bean用来通知其属性变更的标准方式。PropertyChangeSupport类对于任何想要支持属性变更通知其属性变更监听器的bean而言是个便捷的超类。但可惜已有其他超类的类, 比如JComponent, 必须重新实现相应的方法。将PropertyChangeSupport重新实现为一个特质,然后将它混入到java.awt.Point类中

In [12]:
import java.awt.Point
import java.beans.{PropertyChangeSupport, PropertyChangeEvent, PropertyChangeListener}

trait PropertyChange {
    val propertyChangeSupport : PropertyChangeSupport
}

val p = new Point() with PropertyChange {
    val propertyChangeSupport = new PropertyChangeSupport(this)
    propertyChangeSupport.addPropertyChangeListener(new PropertyChangeListener {
        override def propertyChange(evt: PropertyChangeEvent): Unit = {
          println(evt.getPropertyName
            + ": oldValue = " + evt.getOldValue
            + " newValue = " + evt.getNewValue)
        }
      })
}

val newX : Int = 20
p.propertyChangeSupport.firePropertyChange("x", p.getX, newX)
p.move(newX, 30)

x: oldValue = 0.0 newValue = 20


import java.awt.Point

import java.beans.{PropertyChangeSupport, PropertyChangeEvent, PropertyChangeListener}


defined trait PropertyChange
p: Point with wrapper.wrapper.PropertyChange = $sess.cmd11Wrapper$Helper$$anon$1[x=20,y=30]
newX: Int = 20

## 10.6
在Java AWT类库中,我们有一个Container类，一个可以用于各种组件的Component子类。举例来说，Button是一个Component,但Panel是Container。这是一个运转中的组合模式。Swing有JComponent和JContainer，但如果你仔细看的话，你会发现一些奇怪的细节。尽管把其他组件添加到比如JButton中毫无意义,JComponent依然扩展自Container。Swing的设计者们理想情况下应该会更倾向于图10-4中的设计。但在Java中那是不可能的。请解释这是为什么？Scala中如何用特质来设计出这样的效果?

A: Java只能单继承,JContainer不能同时继承自Container和JComponent。Scala可以通过特质解决这个问题.
```scala
trait Component {}
trait JComponent extends Component {}
class JButton extends JComponent {}

trait Container extends Component {}
trait JContainer extends JComponent with Container {}
class JPanel extends JContainer {}
```

## 10.7
市面上有不下数十种关于Scala特质的教程,用的都是些"在叫的狗"啦，"讲哲学的青蛙"啦之类的傻乎乎的例子。阅读和理解这些机巧的继承层级很乏味且对于理解问题没什么帮助,但自己设计一套继承层级就不同了,会很有启发。做一个你自己的关于特质的继承层级，要求体现出叠加在一起的特质,具体的和抽象的方法，以及具体的和抽象的字段

In [13]:
trait Fly{
  def fly() {
    println("flying")
  }

  def flywithnowing()
}

trait Walk{
  def walk(){
    println("walk")
  }
}

class Bird{
  var name:String = _
}

class BlueBird extends Bird with Fly with Walk{
  def flywithnowing() {
    println("BlueBird flywithnowing")
  }
}

val b = new BlueBird()
b.walk()
b.flywithnowing()
b.fly()

walk
BlueBird flywithnowing
flying


defined trait Fly
defined trait Walk
defined class Bird
defined class BlueBird
b: BlueBird = $sess.cmd12Wrapper$Helper$BlueBird@54c72117

## 10.8
在java.io类库中，你可以通过BufferedInputStream修饰器来给输入流增加缓冲机制。用特质来重新实现缓冲。简单起见，重写read方法

In [18]:
import java.io.{FileInputStream, InputStream}

trait Buffering {
    this: InputStream =>

    val BUF_SIZE: Int = 5
    val buf: Array[Byte] = new Array[Byte](BUF_SIZE)
    var bufsize: Int = 0 // 缓存数据大小
    var pos: Int = 0 // 当前位置
    
    override def read(): Int = {
        if (pos >= bufsize) { // 读取数据
            bufsize = this.read(buf, 0, BUF_SIZE)
            if (bufsize <= 0) return bufsize
            pos = 0
        }
        pos += 1 // 移位
        buf(pos - 1) // 返回数据
    }
}

val f = new FileInputStream("myfile.txt") with Buffering
for (i <- 1 to 30) println(f.read() toChar)

T
h
i
s
 
i
s
 
s
c
a
l
a
 
e
x
c
e
r
s
i
e
 
o
f
 
c
h
a
p


import java.io.{FileInputStream, InputStream}


defined trait Buffering
f: FileInputStream with wrapper.wrapper.Buffering = $sess.cmd17Wrapper$Helper$$anon$1@1a4e24

## 10.9
使用本章的日志生成器特质, 给前一个练习中的方案增加日志功能, 要求体现出缓冲的效果

In [23]:
import java.io.{FileInputStream, InputStream}

trait Logger {
    def log(msg: String)
}

trait PrintLogger extends Logger {
    def log(msg: String) = println(msg)
}

trait Buffering {
    this: InputStream with Logger =>

    val BUF_SIZE: Int = 5
    val buf: Array[Byte] = new Array[Byte](BUF_SIZE)
    var bufsize: Int = 0 // 缓存数据大小
    var pos: Int = 0 // 当前位置
    
    override def read(): Int = {
        if (pos >= bufsize) { // 读取数据
            bufsize = this.read(buf, 0, BUF_SIZE)
            if (bufsize <= 0) return bufsize
            log("buffered %d bytes: %s".format(bufsize, buf.map(_.toChar).mkString("")))
            pos = 0
        }
        pos += 1 // 移位
        buf(pos - 1) // 返回数据
    }
}

val f = new FileInputStream("myfile.txt") with Buffering with PrintLogger
for (i <- 1 to 30) println(f.read() toChar)

buffered 5 bytes: This 
T
h
i
s
 
buffered 5 bytes: is sc
i
s
 
s
c
buffered 5 bytes: ala e
a
l
a
 
e
buffered 5 bytes: xcers
x
c
e
r
s
buffered 5 bytes: ie of
i
e
 
o
f
buffered 5 bytes:  chap
 
c
h
a
p


import java.io.{FileInputStream, InputStream}


defined trait Logger
defined trait PrintLogger
defined trait Buffering
f: FileInputStream with wrapper.wrapper.Buffering with wrapper.wrapper.PrintLogger = $sess.cmd22Wrapper$Helper$$anon$1@62dbfad2

## 10.10
实现一个IterableInputStream类，扩展java.io.InputStream并混入Iterable[Byte]特质

In [29]:
import java.io.{FileInputStream, InputStream}

trait IterableInputStream extends InputStream with Iterable[Byte] {
    class InputStreamIterator(outer: IterableInputStream) extends Iterator[Byte] {
        def hasNext: Boolean = outer.available() > 0
        
        def next: Byte = outer.read().toByte
    }
    
    override def iterator: Iterator[Byte] = new InputStreamIterator(this)
}

val fis = new FileInputStream("myfile.txt") with IterableInputStream
val it = fis.iterator
while (it.hasNext) {
    println(it.next() toChar)
}
// fis.close()

T
h
i
s
 
i
s
 
s
c
a
l
a
 
e
x
c
e
r
s
i
e
 
o
f
 
c
h
a
p
t
e
r
 
3
 
i
n
 
j
u
p
y
t
e
r
 
n
o
t
e
b
o
o
k
 
s
c
a
l
a
 
e
n
v
i
r
o
n
m
e
n
t




import java.io.{FileInputStream, InputStream}


defined trait IterableInputStream
fis: FileInputStream with IterableInputStream = cmd28Wrapper()
it: Iterator[Byte] = empty iterator